In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_backup = pd.read_csv('../csvs/deliveries.csv')
data = data_backup.copy()
data.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [3]:
original_df = data

original_df['start_date'] = pd.to_datetime(original_df['start_date'])


grouped_df = original_df.groupby(['match_id', 'innings', 'venue'])


result_df = grouped_df.agg({
    'batting_team': 'first',
    'bowling_team': 'first',
    'wides': 'sum',
    'noballs': 'sum',
    'byes': 'sum',
    'legbyes': 'sum',
    'runs_off_bat': 'sum',
    'ball': 'max',
    'wicket_type': lambda x: x.notnull().sum(),
    'start_date': 'first'
}).reset_index()


result_df.columns = ['match_id', 'innings', 'venue', 'batting_team', 'bowling_team',
                      'total_wides', 'total_noballs', 'total_byes', 'total_legbyes', 'total_runs_per_innings_match',
                       'last_ball', 'total_wickets', 'date']


result_df['total_overs_played'] = result_df['last_ball'].apply(lambda x: min((int(x) + round((x % 1) * 10, 4) / 6), 50.0))


result_df[['total_wides', 'total_noballs', 'total_byes', 'total_legbyes']] = result_df[['total_wides', 'total_noballs', 'total_byes', 'total_legbyes']].astype(int)


result_df['total_runs_per_innings_match'] += result_df['total_wides'] + result_df['total_noballs'] + result_df['total_byes'] + result_df['total_legbyes']

result_df.drop(['last_ball'], axis=1, inplace=True)

In [4]:
result_df.head()

,match_id,innings,venue,batting_team,bowling_team,total_wides,total_noballs,total_byes,total_legbyes,total_runs_per_innings_match,total_wickets,date,total_overs_played
0,1,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,6,0,0,0,282,9,2023-10-05,50.000000
1,1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,3,0,4,1,283,1,2023-10-05,36.333333
2,2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,8,1,0,0,286,10,2023-10-06,49.000000
3,2,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,9,0,0,0,205,10,2023-10-06,41.166667
4,3,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,8,0,0,0,156,10,2023-10-07,37.333333


In [5]:
final_df = result_df[result_df['innings'] == 2]

In [6]:
#extract run for innings 1 from result_df and add it to final_df
first_innings = result_df[result_df['innings'] == 1]
first_innings = first_innings[['match_id', 'total_runs_per_innings_match']]
first_innings.columns = ['match_id', 'total_runs_in_innings1']
final_df = pd.merge(final_df, first_innings, on='match_id')
final_df.head()

,match_id,innings,venue,batting_team,bowling_team,total_wides,total_noballs,total_byes,total_legbyes,total_runs_per_innings_match,total_wickets,date,total_overs_played,total_runs_in_innings1
0,1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,3,0,4,1,283,1,2023-10-05,36.333333,282
1,2,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,9,0,0,0,205,10,2023-10-06,41.166667,286
2,3,2,"Himachal Pradesh Cricket Association Stadium, ...",Bangladesh,Afghanistan,4,1,0,3,158,4,2023-10-07,34.666667,156
3,4,2,"Arun Jaitley Stadium, Delhi",Sri Lanka,South Africa,14,1,0,5,326,10,2023-10-07,44.833333,428
4,5,2,"MA Chidambaram Stadium, Chepauk, Chennai",India,Australia,5,1,1,1,201,4,2023-10-08,41.333333,199


In [7]:
final_df.drop(['match_id','total_wickets','innings','total_wides','total_noballs','total_noballs','total_byes','total_legbyes','date'],axis=1,inplace=True)

In [8]:
final_df.head()

,venue,batting_team,bowling_team,total_runs_per_innings_match,total_overs_played,total_runs_in_innings1
0,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,283,36.333333,282
1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,205,41.166667,286
2,"Himachal Pradesh Cricket Association Stadium, ...",Bangladesh,Afghanistan,158,34.666667,156
3,"Arun Jaitley Stadium, Delhi",Sri Lanka,South Africa,326,44.833333,428
4,"MA Chidambaram Stadium, Chepauk, Chennai",India,Australia,201,41.333333,199


In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

mapping = {}
categorical_columns = ['venue','batting_team','bowling_team']

for column in categorical_columns:
    final_df[column] = le.fit_transform(final_df[column])
    mapping[column] = dict(zip(le.classes_, le.transform(le.classes_)))

In [10]:
mapping

{'venue': {'Arun Jaitley Stadium, Delhi': 0,
  'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow': 1,
  'Eden Gardens, Kolkata': 2,
  'Himachal Pradesh Cricket Association Stadium, Dharamsala': 3,
  'M Chinnaswamy Stadium, Bengaluru': 4,
  'MA Chidambaram Stadium, Chepauk, Chennai': 5,
  'Maharashtra Cricket Association Stadium, Pune': 6,
  'Narendra Modi Stadium, Ahmedabad': 7,
  'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 8,
  'Wankhede Stadium, Mumbai': 9},
 'batting_team': {'Afghanistan': 0,
  'Australia': 1,
  'Bangladesh': 2,
  'England': 3,
  'India': 4,
  'Netherlands': 5,
  'New Zealand': 6,
  'Pakistan': 7,
  'South Africa': 8,
  'Sri Lanka': 9},
 'bowling_team': {'Afghanistan': 0,
  'Australia': 1,
  'Bangladesh': 2,
  'England': 3,
  'India': 4,
  'Netherlands': 5,
  'New Zealand': 6,
  'Pakistan': 7,
  'South Africa': 8,
  'Sri Lanka': 9}}

In [11]:
X=final_df.drop('total_runs_per_innings_match',axis='columns')
y=final_df['total_runs_per_innings_match']

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=42)

size_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = size_scaler.transform(X_train)
X_test_scaled = size_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((28, 5), (4, 5))

In [13]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8959206373356593

In [14]:
#predict y_test
y_pred = model.predict(X_test)

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler


model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer with 1 unit and linear activation
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=1, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/500
22/22 [==============================] - 1s 9ms/step - loss: 45701.7539 - mae: 202.9442 - val_loss: 35231.5469 - val_mae: 175.8212
Epoch 2/500
22/22 [==============================] - 0s 3ms/step - loss: 32452.7500 - mae: 169.0838 - val_loss: 21534.9531 - val_mae: 131.4732
Epoch 3/500
22/22 [==============================] - 0s 2ms/step - loss: 18642.2871 - mae: 125.1509 - val_loss: 9458.1133 - val_mae: 78.9838
Epoch 4/500
22/22 [==============================] - 0s 3ms/step - loss: 8515.3770 - mae: 80.7132 - val_loss: 4898.5122 - val_mae: 61.9097
Epoch 5/500
22/22 [==============================] - 0s 3ms/step - loss: 5354.5854 - mae: 59.6622 - val_loss: 5302.8208 - val_mae: 62.9413
Epoch 6/500
22/22 [==============================] - 0s 2ms/step - loss: 4930.6167 - mae: 57.6742 - val_loss: 4923.8633 - val_mae: 62.0530
Epoch 7/500
22/22 [==============================] - 0s 3ms/step - loss: 5137.7427 - mae: 58.2100 - val_loss: 5053.8750 - val_mae: 62.2053
Epoch 8/500
22/22

In [16]:
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
print(f"Training MAE: {train_mae}")

# Evaluate on the test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test MAE: {test_mae}")

1/1 [==============================] - 0s 146ms/step - loss: 4278.5161 - mae: 54.4699
Training MAE: 54.46991729736328
1/1 [==============================] - 0s 14ms/step - loss: 2442.9072 - mae: 44.5101
Test MAE: 44.51009750366211


In [17]:
y_pred = model.predict(X_test)
y_pred

1/1 [==============================] - 0s 65ms/step


array([[198.74583],
       [218.85696],
       [129.8005 ],
       [280.2703 ]], dtype=float32)

In [18]:
y_test

29    242
15    139
24    160
17    305
Name: total_runs_per_innings_match, dtype: int64

In [19]:
import pickle as pkl

In [20]:
pkl.dump(model, open('../pickles/inning2_run_predictor', 'wb'))